## Download gene annotations v42 (latest) - of CHR region

In [13]:
!wget -O /s/project/first_last_exon/Data/input_data/gencode.v42.annotation.gtf.gz https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_42/gencode.v42.annotation.gtf.gz

--2023-01-10 21:59:24--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_42/gencode.v42.annotation.gtf.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49548290 (47M) [application/x-gzip]
Saving to: ‘/s/project/first_last_exon/Data/input_data/gencode.v42.annotation.gtf.gz’

/s/project/first_la 100%[===================>]  47.25M  1.85MB/s    in 23s     

2023-01-10 21:59:48 (2.01 MB/s) - ‘/s/project/first_last_exon/Data/input_data/gencode.v42.annotation.gtf.gz’ saved [49548290/49548290]



In [16]:
# unzip annotations file
!gzip -d /s/project/first_last_exon/Data/input_data/gencode.v42.annotation.gtf.gz

## Step 1

### Run annotation script

In [3]:
# see annotations file content

import pyranges as pr

# as PyRanges-object
gr = pr.read_gtf("/s/project/first_last_exon/Data/input_data/gencode.v42.annotation.gtf")


In [4]:
gr

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_support_level,havana_transcript,exon_number,exon_id,hgnc_id,havana_gene,ont,protein_id,ccdsid,artif_dupl
0,chr1,HAVANA,gene,11868,14409,.,+,.,ENSG00000290825.1,lncRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,11868,14409,.,+,.,ENSG00000290825.1,lncRNA,...,1,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,11868,12227,.,+,.,ENSG00000290825.1,lncRNA,...,1,OTTHUMT00000362751.1,1,ENSE00002234944.1,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,HAVANA,exon,12612,12721,.,+,.,ENSG00000290825.1,lncRNA,...,1,OTTHUMT00000362751.1,2,ENSE00003582793.1,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,exon,13220,14409,.,+,.,ENSG00000290825.1,lncRNA,...,1,OTTHUMT00000362751.1,3,ENSE00002312635.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3413836,chrY,HAVANA,exon,57214349,57214397,.,-,.,ENSG00000227159.8_PAR_Y,unprocessed_pseudogene,...,NA,OTTHUMT00000058841.1,1,ENSE00002072208.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN
3413837,chrY,HAVANA,exon,57213879,57213964,.,-,.,ENSG00000227159.8_PAR_Y,unprocessed_pseudogene,...,NA,OTTHUMT00000058841.1,2,ENSE00002046926.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN
3413838,chrY,HAVANA,exon,57213525,57213602,.,-,.,ENSG00000227159.8_PAR_Y,unprocessed_pseudogene,...,NA,OTTHUMT00000058841.1,3,ENSE00002021169.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN
3413839,chrY,HAVANA,exon,57213203,57213357,.,-,.,ENSG00000227159.8_PAR_Y,unprocessed_pseudogene,...,NA,OTTHUMT00000058841.1,4,ENSE00002036959.1,HGNC:37115,OTTHUMG00000022678.1,PGO:0000005,NaN,NaN,NaN


In [1]:
%cd HITindex/hitindex

/data/nasif12/home_if12/afi/first_last_exon/HITindex/hitindex


In [ ]:
!python HITindex_annotate.py --gtf /s/project/first_last_exon/Data/input_data/gencode.v42.annotation.gtf --ss3buffer 50 --ss5buffer 20 --outfile /s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_1/metaexons.bed

Read output file

In [10]:
import pandas as pd
import os 

print("Output files\n", os.listdir("/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_1/"))

file_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_1/metaexons.bed"

file_read = pd.read_csv(file_path, sep='\t', names=('chr', 'start', 'stop', 'meta', 'count', 'strand' ), header=None, index_col=None)

Output files
 ['metaexons.bed_ss3-50ss5-20.buffer', 'metaexons.bed', 'metaexons.bed_constituents']


In [11]:
file_read

,chr,start,stop,meta,count,strand
0,chr1,11869,12227,chr1:11869-12227;ENSG00000290825.1;TXPT:1;FE:0...,0,+
1,chr1,12613,12721,chr1:12613-12721;ENSG00000290825.1;TXPT:1;FE:0...,0,+
2,chr1,13221,14409,chr1:13221-14409;ENSG00000290825.1;TXPT:1;FE:0...,0,+
3,chr1,12010,12057,chr1:12010-12057;ENSG00000223972.6;TXPT:1;FE:0...,0,+
4,chr1,12179,12227,chr1:12179-12227;ENSG00000223972.6;TXPT:1;FE:0...,0,+
...,...,...,...,...,...,...
358272,chrM,14149,14673,chrM:14149-14673;ENSG00000198695.2;TXPT:1;FE:0...,0,-
358273,chrM,14674,14742,chrM:14674-14742;ENSG00000210194.1;TXPT:1;FE:0...,0,-
358274,chrM,14747,15887,chrM:14747-15887;ENSG00000198727.2;TXPT:1;FE:0...,0,+
358275,chrM,15888,15953,chrM:15888-15953;ENSG00000210195.2;TXPT:1;FE:0...,0,+


## Step 2

### Run classification script on all CHR bam files

In [53]:
class args:
    def __init__(self, junctionReads, bam, juncbam, readstrand, HITindex, bed, classify, calculatePSI, metricsID, outname):
        self.junctionReads = junctionReads
        self.bam = bam
        self.juncbam = juncbam
        self.readstrand = readstrand
        self.HITindex = HITindex
        self.bed = bed
        self.classify = classify
        self.calculatePSI = calculatePSI
        self.metricsID = metricsID
        self.outname = outname
        self.readtype = 'paired'
        self.overlap = 10
        self.readnum = 2
        self.parameters = "HIT_identity_parameters.txt"
        self.bootstrap = 1000
        self.edge  = False

In [54]:
import os
import subprocess
import HITindex_classify_mainfn as hitmain

datafolder_path = "/s/project/first_last_exon/Data/input_data/rna-seq/"
output_folder_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/"
bed_file_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_1/metaexons.bed_ss3-50ss5-20.buffer"
bam_files_list = [f for f in os.listdir(datafolder_path) if f.endswith("_chr1.bam")]
print(len(bam_files_list), "files detected")

for i, f in enumerate(bam_files_list):
    sample_name = f.split("_chr1.bam")[0]
    junctionReads = True
    bam = os.path.join(datafolder_path, f)
    juncbam = os.path.join(output_folder_path, sample_name + ".junctions.bam") 
    readstrand = "fr-unstrand"
    HITindex = True
    bed = bed_file_path 
    classify = True
    calculatePSI = True
    outname = os.path.join(output_folder_path, sample_name + "_HITindex")
    arguments = args(junctionReads, bam, juncbam, readstrand, HITindex, bed, classify, calculatePSI, metricsID, outname)
    print("Computing sample: {}, {}/{}".format(sample_name, i+1, len(bam_files_list)))
    hitmain.main(arguments)
    print("Sample {} is done!".format(sample_name))
          

# !python HITindex_classify.py --junctionReads --bam /s/project/first_last_exon/Data/input_data/rna-seq/HG00096.1.M_111124_6_chr1.bam --juncbam /s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1.junctions.bam --readstrand fr-unstrand --HITindex --bed /s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_1/metaexons.bed_ss3-50ss5-20.buffer --classify --calculatePSI --metricsID sampleHITindex.exon --outname /s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1_HITindex   

100 files detected
Computing sample: NA20796.1.M_120209_2, 1/100
Obtaining junctions...
... junctions obtained.
Calculating metric...
... 358277 exons processed in 62696 genes.
... got exons...
NOTE: unstranded reads
... got junctions...
... got counts...
HIT for exons...1000..2000..3000..4000..5000..6000..7000..8000..9000..10000..11000..12000..13000..14000..15000..16000..17000..18000..19000..20000..21000..22000..23000..24000..25000..26000..27000..28000..29000..30000..31000..32000..33000..34000..35000..36000..37000..38000..39000..40000..41000..42000..43000..44000..45000..46000..47000..48000..49000..50000..51000..52000..53000..54000..55000..56000..57000..58000..59000..60000..61000..62000..63000..64000..65000..66000..67000..68000..69000..70000..71000..72000..73000..74000..75000..76000..77000..78000..79000..80000..81000..82000..83000..84000..85000..86000..87000..88000..89000..90000..91000..92000..93000..94000..95000..96000..97000..98000..99000..100000..101000..102000..103000..104000..1050

/data/ouga/home/ag_gagneur/afi/.conda/envs/first_last_exon/lib/python3.6/site-packages/pymc3/data.py:307: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.shared = theano.shared(data[in_memory_slc])


Finished [100%]: Average Loss = 3,240.5


... generative model calculated.
Classifying exons...
...read exons.
...read parameters.
...calculated significance.
... exons identified.
Calculating PSI...


/data/nasif12/home_if12/afi/first_last_exon/HITindex/hitindex/HITindex_classify_mainfn.py:505: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afehere['sumR-L'] = sum(RLlist)
/data/nasif12/home_if12/afi/first_last_exon/HITindex/hitindex/HITindex_classify_mainfn.py:506: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afehere['AFEPSI'] = AFEPSI
/data/nasif12/home_if12/afi/first_last_exon/HITindex/hitindex/HITindex_classify_mainfn.py:516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

...

Finished [100%]: Average Loss = 2,966.3


... generative model calculated.
Classifying exons...
...read exons.
...read parameters.
...calculated significance.
... exons identified.
Calculating PSI...
... PSI calculated.
Sample HG00189.1.M_111124_5 is done!
Computing sample: NA11994.2.M_111216_7, 12/100
Obtaining junctions...
... junctions obtained.
Calculating metric...
... 358277 exons processed in 62696 genes.
... got exons...
NOTE: unstranded reads
... got junctions...
... got counts...
HIT for exons...1000..2000..3000..4000..5000..6000..7000..8000..9000..10000..11000..12000..13000..14000..15000..16000..17000..18000..19000..20000..21000..22000..23000..24000..25000..26000..27000..28000..29000..30000..31000..32000..33000..34000..35000..36000..37000..38000..39000..40000..41000..42000..43000..44000..45000..46000..47000..48000..49000..50000..51000..52000..53000..54000..55000..56000..57000..58000..59000..60000..61000..62000..63000..64000..65000..66000..67000..68000..69000..70000..71000..72000..73000..74000..75000..76000..77000..7

Finished [100%]: Average Loss = 3,153.5


... generative model calculated.
Classifying exons...
...read exons.
...read parameters.


KeyboardInterrupt: 

CompletedProcess(args='ls', returncode=0)


In [9]:
# Read junction bam file
import pysam

samfile = pysam.AlignmentFile("/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1.junctions_read2.bam", "rb")
for read in samfile.fetch('chr1', 100, 120):
    print(read)

samfile.close()



header_file_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1.junctions.bam_header.txt"
with open(header_file_path, "r") as f:
    lines = f.readlines()
    print(lines)

['@HD\tVN:1.3\n', '@SQ\tSN:chr1\tLN:249250621\n', '@SQ\tSN:chr10\tLN:135534747\n', '@SQ\tSN:chr11\tLN:135006516\n', '@SQ\tSN:chr12\tLN:133851895\n', '@SQ\tSN:chr13\tLN:115169878\n', '@SQ\tSN:chr14\tLN:107349540\n', '@SQ\tSN:chr15\tLN:102531392\n', '@SQ\tSN:chr16\tLN:90354753\n', '@SQ\tSN:chr17\tLN:81195210\n', '@SQ\tSN:chr18\tLN:78077248\n', '@SQ\tSN:chr19\tLN:59128983\n', '@SQ\tSN:chr2\tLN:243199373\n', '@SQ\tSN:chr20\tLN:63025520\n', '@SQ\tSN:chr21\tLN:48129895\n', '@SQ\tSN:chr22\tLN:51304566\n', '@SQ\tSN:chr3\tLN:198022430\n', '@SQ\tSN:chr4\tLN:191154276\n', '@SQ\tSN:chr5\tLN:180915260\n', '@SQ\tSN:chr6\tLN:171115067\n', '@SQ\tSN:chr7\tLN:159138663\n', '@SQ\tSN:chr8\tLN:146364022\n', '@SQ\tSN:chr9\tLN:141213431\n', '@SQ\tSN:chrX\tLN:155270560\n', '@SQ\tSN:chrY\tLN:59373566\n', '@RG\tID:0\tPG:GEM\tPL:ILLUMINA\tSM:0\n', '@PG\tID:GEM\tPN:gem-2-sam\tVN:1.306\n']


In [11]:
# Read output files
import pandas as pd
exon_file_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1_HITindex.exon"
AFEPSI_file_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1_HITindex.AFEPSI"
ALEPSI_file_path = "/s/project/first_last_exon/Data/output_data/HITindex/rna_seq/step_2/HG00096.1.M_111124_6_chr1_HITindex.ALEPSI"
exon_df = pd.read_csv(exon_file_path, sep='\t')
AFEPSI_df = pd.read_csv(AFEPSI_file_path, sep='\t')
ALEPSI_df = pd.read_csv(ALEPSI_file_path, sep='\t')

In [19]:
exon_df.head()

,exon,gene,strand,nTXPT,nFE,nINTERNAL,nLE,nSINGLE,nUP,nDOWN,...,PofIL,downstream_fraction,HIT_postmean,CI_75,CI_90,CI_95,pval_CI,pval_internal,ID,ID_position
0,chr1:15005-15038,ENSG00000227232.5,-,1,0,0,0,0,11,8,...,0.09185,0.49122,0.01756,"-0.158,0.368","-0.263,0.474","-0.263,0.579",0.773,0.322,internal,last
1,chr1:15796-15947,ENSG00000227232.5,-,1,0,0,0,0,5,25,...,0.00012,0.77439,-0.54878,"-0.8,-0.467","-0.867,-0.4","-0.867,-0.333",0.015,0.000,FirstInternal_high,FirstInternal_high
2,chr1:16607-16765,ENSG00000227232.5,-,1,0,0,0,0,0,6,...,0.00104,0.95758,-0.91515,"-1.0,-0.333","-1.0,-0.333","-1.0,-0.325",0.023,0.017,first,first
3,chr1:16858-17055,ENSG00000227232.5,-,1,0,0,0,0,9,0,...,0.08771,0.01269,0.97463,"0.556,1.0","0.556,1.0","0.333,1.0",0.006,0.004,last,last
4,chr1:17233-17368,ENSG00000227232.5,-,1,0,0,0,0,58,6,...,0.99959,0.10753,0.78493,"0.688,0.875","0.656,0.906","0.625,0.938",0.000,0.000,InternalLast_high,InternalLast_high


In [18]:
exon_df.iloc[:,:12]

,exon,gene,strand,nTXPT,nFE,nINTERNAL,nLE,nSINGLE,nUP,nDOWN,HITindex,dist_to_TSS
0,chr1:15005-15038,ENSG00000227232.5,-,1,0,0,0,0,11,8,0.158,9853
1,chr1:15796-15947,ENSG00000227232.5,-,1,0,0,0,0,5,25,-0.667,8944
2,chr1:16607-16765,ENSG00000227232.5,-,1,0,0,0,0,0,6,-1.000,8126
3,chr1:16858-17055,ENSG00000227232.5,-,1,0,0,0,0,9,0,1.000,7836
4,chr1:17233-17368,ENSG00000227232.5,-,1,0,0,0,0,58,6,0.812,7523
...,...,...,...,...,...,...,...,...,...,...,...,...
1261,chr1:247353953-247354287,ENSG00000289234.2,+,16,0,0,0,0,0,24,-1.000,0
1262,chr1:247374007-247374143,ENSG00000289234.2,+,16,0,0,0,0,9,0,1.000,20054
1263,chr1:247374007-247374143,ENSG00000289728.1,+,1,0,0,0,0,9,0,1.000,0
1264,chr1:247491068-247492415,ENSG00000290684.1,+,2,0,0,0,0,7,10,-0.176,0


In [21]:
exon_df.to_csv( "./exon.csv")

In [22]:
AFEPSI_df.head()

,gene,exon,strand,nTXPT,nFE,nUP,nDOWN,HITindex,sumR-L,AFEPSI
0,ENSG00000227232.5,chr1:15796-15947,-,1,0,5,25,-0.667,86,0.232558
1,ENSG00000227232.5,chr1:16607-16765,-,1,0,0,6,-1.000,86,0.069767
2,ENSG00000227232.5,chr1:17606-17742,-,1,0,6,60,-0.818,86,0.627907
3,ENSG00000227232.5,chr1:17915-18061,-,1,0,0,6,-1.000,86,0.069767
4,ENSG00000187642.10,chr1:976499-976641,-,4,0,0,19,-1.000,19,1.000000
